# RFM Analizi ile Müşteri Segmentasyonu

## İş Problemi: İngiltere merkezli perakende şirketi müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istemektedir. Ortak davranışlar sergileyen müşteri segmentleri özelinde pazarlama çalışmaları yapmanın gelir artışı sağlayacağını düşünmektedir. Segmentlere ayırmak için RFM analizi kullanılacaktır.


## Veri Seti Hikayesi: Online Retail II isimli veri seti İngiltere merkezli bir perakende şirketinin 01/12/2009 - 09/12/2011 tarihleri arasındaki online satış işlemlerini içeriyor. Şirketin ürün kataloğunda hediyelik eşyalar yer almaktadır ve çoğu müşterisinin toptancı olduğu bilgisi mevcuttur.

### Görev 1: Veriyi Anlama ve Hazırlama


In [15]:
import pandas as pd
import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.width',1000)

Adım 1: Online Retail II excelindeki 2010-2011 verisini okuyunuz. Oluşturduğunuz dataframe’in kopyasını oluşturunuz

In [19]:
df_ = pd.read_excel("/content/online_retail_II.xlsx")

In [20]:
df = df_.copy()

In [21]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom


Adım 2: Veri setinin betimsel istatistiklerini inceleyiniz.

In [22]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,525461.00,10.34,107.42,-9600.00,1.00,3.00,10.00,19152.00
Price,525461.00,4.69,146.13,-53594.36,1.25,2.10,4.21,25111.09
Customer ID,417534.00,15360.65,1680.81,12346.00,13983.00,15311.00,16799.00,18287.00


Adım 3: Veri setinde eksik gözlem var mı? Varsa hangi değişkende kaç tane eksik gözlem vardır?

In [24]:
df.isnull().values.any()

True

In [25]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

Adım 4: Eksik gözlemleri veri setinden çıkartınız. Çıkarma işleminde ‘inplace=True’ parametresini kullanınız

In [26]:
df.dropna(inplace=True)

In [27]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

Adım 5: Eşsiz ürün sayısı kaçtır?

In [28]:
df.nunique()

Invoice        23587
StockCode       4031
Description     4459
Quantity         500
InvoiceDate    21786
Price            664
Customer ID     4383
Country           37
dtype: int64

Adım 6: Hangi üründen kaçar tane vardır?

In [29]:
df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    3245
REGENCY CAKESTAND 3 TIER              1872
STRAWBERRY CERAMIC TRINKET BOX        1536
ASSORTED COLOUR BIRD ORNAMENT         1376
HOME BUILDING BLOCK WORD              1229
Name: Description, dtype: int64

Adım 7: En çok sipariş edilen 5 ürünü çoktan aza doğru sıralayınız

In [30]:
df.groupby("Description").agg({"Quantity":"sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,55861
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54274
BROCADE RING PURSE,47430
PACK OF 72 RETRO SPOT CAKE CASES,44507
ASSORTED COLOUR BIRD ORNAMENT,44120


Adım 8: Faturalardaki ‘C’ iptal edilen işlemleri göstermektedir. İptal edilen işlemleri veri setinden çıkartınız.

In [31]:
df = df[~df["Invoice"].str.contains("C", na=False)]

Adım 9: Fatura başına elde edilen toplam kazancı ifade eden ‘TotalPrice’ adında bir değişken oluşturunuz


In [ ]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [33]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom,30.00


### Görev 2: RFM Metriklerinin Hesaplanması

Adım 1: Recency, Frequency ve Monetary tanımlarını yapınız.

Adım 2: Müşteri özelinde Recency, Frequency ve Monetary metriklerini groupby, agg ve lambda ile hesaplayınız.

Adım 3: Hesapladığınız metrikleri rfm isimli bir değişkene atayınız.

Adım 4: Oluşturduğunuz metriklerin isimlerini recency, frequency ve monetary olarak değiştiriniz



In [34]:
df["InvoiceDate"].max()

Timestamp('2010-12-09 20:01:00')

In [35]:
today_date = dt.datetime(2010,12,11)

In [36]:
rfm = df.groupby("Customer ID").agg({"InvoiceDate":lambda InvoiceDate:(today_date-InvoiceDate.max()).days,
                                     "Invoice":lambda Invoice:Invoice.nunique(),
                                     "TotalPrice":lambda TotalPrice:TotalPrice.sum()})

In [37]:
rfm.columns = ["recency","frequency","monetary"]

In [38]:
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12346.00,165,11,372.86
12347.00,3,2,1323.32
12348.00,74,1,222.16
12349.00,43,3,2671.14
12351.00,11,1,300.93


In [39]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4314.00,91.27,96.94,1.00,18.00,53.00,136.00,374.00
frequency,4314.00,4.45,8.17,1.00,1.00,2.00,5.00,205.00
monetary,4314.00,2047.29,8912.52,0.00,307.95,705.55,1722.80,349164.35


In [40]:
rfm = rfm[rfm["monetary"]>0]

In [41]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.00,91.17,96.86,1.00,18.00,53.00,136.00,374.00
frequency,4312.00,4.46,8.17,1.00,1.00,2.00,5.00,205.00
monetary,4312.00,2048.24,8914.48,2.95,307.99,706.02,1723.14,349164.35


In [42]:
rfm.shape

(4312, 3)

### Görev 3: RFM Skorlarının Oluşturulması ve Tek bir Değişkene Çevrilmesi

Adım 1: Recency, Frequency ve Monetary metriklerini qcut yardımı ile 1-5 arasında skorlara çeviriniz.

Adım 2: Bu skorları recency_score, frequency_score ve monetary_score olarak kaydediniz.

In [43]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5,4,3,2,1])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1,2,3,4,5])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])

In [44]:
rfm.head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score
Customer ID,,,,,,
12346.00,165,11,372.86,2,2,5
12347.00,3,2,1323.32,5,4,2
12348.00,74,1,222.16,2,1,1
12349.00,43,3,2671.14,3,5,3
12351.00,11,1,300.93,5,2,1


Adım 3: recency_score ve frequency_score’u tek bir değişken olarak ifade ediniz ve RF_SCORE olarak kaydediniz.

In [45]:
rfm["RF_SCORE"] = (rfm["recency_score"].astype(str)+(rfm["frequency_score"].astype(str)))

In [46]:
rfm.head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RF_SCORE
Customer ID,,,,,,,
12346.00,165,11,372.86,2,2,5,25
12347.00,3,2,1323.32,5,4,2,52
12348.00,74,1,222.16,2,1,1,21
12349.00,43,3,2671.14,3,5,3,33
12351.00,11,1,300.93,5,2,1,51


### Görev 4: RF Skorunun Segment Olarak Tanımlanması


Adım 1: Oluşturulan RF skorları için segment tanımlamaları yapınız.

Adım 2: seg_map yardımı ile skorları segmentlere çeviriniz.

In [47]:
seg_map = {r"[1-2][1-2]":"hibernating",
           r"[1-2][3-4]":"at-Risk",
           r"[1-2]5":"cant_loose",
           r"3[1-2]":"about_to_sleep",
           r"33":"need_attention",
           r"[3-4][4-5]":"loyal_customers",
           r"41":"promosing",
           r"51":"new_customers",
           r"[4-5][2-3]":"potential_loyalists",
           r"5[4-5]":"champions"
}

In [48]:
rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)

In [49]:
rfm.head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RF_SCORE,segment
Customer ID,,,,,,,,
12346.00,165,11,372.86,2,2,5,25,cant_loose
12347.00,3,2,1323.32,5,4,2,52,potential_loyalists
12348.00,74,1,222.16,2,1,1,21,hibernating
12349.00,43,3,2671.14,3,5,3,33,need_attention
12351.00,11,1,300.93,5,2,1,51,new_customers


### Görev 5: Aksiyon Zamanı !

Adım 1: Segmentleri hem aksiyon kararları açısından hemde segmentlerin yapısı açısından(ortalama
RFM değerleri) yorumlayınız.

In [50]:
rfm[["segment","recency","frequency","monetary"]].groupby("segment").agg(["mean","count"])

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep        53.82   343      1.20   343   441.32   343
at-Risk              152.16   611      3.07   611  1188.88   611
cant_loose           124.12    77      9.12    77  4099.45    77
champions              7.12   663     12.55   663  6852.26   663
hibernating          213.89  1015      1.13  1015   403.98  1015
loyal_customers       36.29   742      6.83   742  2746.07   742
need_attention        53.27   207      2.45   207  1060.36   207
new_customers          8.58    50      1.00    50   386.20    50
potential_loyalists   18.79   517      2.02   517   729.51   517
promosing             25.75    87      1.00    87   367.09    87

Adım 2: "Loyal Customers" sınıfına ait customer ID'leri seçerek excel çıktısını alınız.

In [51]:
new_df = pd.DataFrame()
new_df["Loyal Customers ID"] = rfm[rfm["segment"]=="loyal_customers"].index
new_df["Loyal Customers ID"] = new_df["Loyal Customers ID"].astype(int)
new_df.to_csv("loyal_customers_id.csv")

In [ ]:
# Bütün rfm'i dışarı çıkarmak için;
# rfm.to_csv("rfm.csv")